# 📍CNN


 ## Visdom Example

In [10]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [4]:
# visdom server를 켜고 실행해야 import error가 발생하지 않는다.
# 명령 프롬프트에 'visdom' 입력
import torch
import visdom
vis = visdom.Visdom()

Setting up a new session...


#### Text

In [5]:
vis.text("Hello World!", env="main")

'window_3a6a053a6a4ba0'

#### Image

In [6]:
a = torch.randn(3, 200, 200)
vis.image(a)

'window_3a6a053c599046'

#### Images

In [7]:
vis.images(torch.Tensor(3, 3, 28, 28))

'window_3a6a053e7e9a40'

### 1. example (using MNIST and CIFAR10)

In [8]:
import torchvision

In [11]:
MNIST = dsets.MNIST(root='./MNIST_data', train=True, transform=torchvision.transforms.ToTensor(), download=True)
cifar10 = dsets.CIFAR10(root='./cifar10', train=True, transform=torchvision.transforms.ToTensor(), download=True)

Files already downloaded and verified


#### CIFAR10

In [12]:
data = cifar10.__getitem__(0)
print(data[0].shape)
vis.images(data[0], env='main')

torch.Size([3, 32, 32])


'window_3a6a055800e1a8'

#### MNIST

In [13]:
data = MNIST.__getitem__(0)
print(data[0].shape)
vis.images(data[0], env='main')

torch.Size([1, 28, 28])


'window_3a6a0559dcb6fa'

#### Check Dataset

In [14]:
data_loader = torch.utils.data.DataLoader(dataset=MNIST,
                                          batch_size=32,
                                          shuffle=False)

In [15]:
for num, value in enumerate(data_loader):
    value = value[0]
    print(value.shape)
    vis.images(value)
    break

torch.Size([32, 1, 28, 28])


In [16]:
vis.close(env='main')

''

### 2. Line Plot
그래프 표현

In [17]:
Y_data = torch.randn(5)
plt = vis.line(Y=Y_data)

x축을 선언해주지 않으면 무조건 0-1 사이의 값으로 선언된다.

In [18]:
X_data = torch.Tensor([1, 2, 3, 4, 5])
plt = vis.line(Y=Y_data, X=X_data)

#### Line Update

In [24]:
Y_append = torch.randn(1)
X_append = torch.Tensor([6])

vis.line(Y=Y_append, X=X_append, win=plt, update='append')

'window_3a6a057b820efc'

#### multiple Line on Single Windows

In [25]:
num = torch.Tensor(list(range(0, 10)))
print(num.shape)
num = num.view(-1, 1)
print(num.shape)
num = torch.cat((num, num), dim=1)
print(num.shape)

plt = vis.line(Y=torch.randn(10, 2), X=num)

torch.Size([10])
torch.Size([10, 1])
torch.Size([10, 2])


In [26]:
vis.close()

''

#### Line info
line info는 dictionary 형태로 값을 넣어주면 된다.

In [30]:
a = dict(title="Test", legend='X')

In [31]:
a

{'title': 'Test', 'legend': 'X'}

In [32]:
plt = vis.line(Y=Y_data, X=X_data, opts=dict(title="Test", showlegend=True))

In [33]:
plt = vis.line(Y=Y_data, X=X_data, opts=dict(title="Test", legend=['1번'], showlegend=True))

In [34]:
plt = vis.line(Y=torch.randn(10, 2), X=num,opts=dict(title='Test', legend=['1번', '2번']))

### 3. make function for update line

In [35]:
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value are Tensor'''
    vis.line(X=num,
             Y=loss_value,
             win=loss_plot,
             update='append'
             )

In [36]:
plt = vis.line(Y=torch.Tensor(1).zero_())

for i in range(500):
    loss = torch.randn(1) * i
    loss_tracker(plt, loss, torch.Tensor([i]))

In [37]:
vis.close(env='main')

''

### 4. MNIST-CNN with Visdom

In [38]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [39]:
import visdom

vis = visdom.Visdom()
vis.close(env='main')

Setting up a new session...


''

#### define loss_tracker

In [40]:
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value are Tensor'''
    vis.line(X=num,
             Y=loss_value,
             win=loss_plot,
             update='append'
             )

In [41]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [42]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [43]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [44]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [45]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc = nn.Linear(7*7*64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        
        return out

In [46]:
model = CNN().to(device)

In [47]:
# 모델 테스트
value = torch.Tensor(1, 1, 28, 28).to(device)
print((model(value)).shape)

torch.Size([1, 10])


In [48]:
# loss & optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#### make plot

In [49]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(), opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

#### train with loss_tracker

In [50]:
# training
total_batch = len(data_loader) # 전체 배치 사이즈

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device) # input
        Y = Y.to(device) # label
        
        optimizer.zero_grad() # 이것을 빼먹으면 학습이 아예 안된다.
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print("[Epoch {}] cost = {}".format(epoch+1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
print("Learning Finished!")

[Epoch 1] cost = 0.22560782730579376
[Epoch 2] cost = 0.0629417672753334
[Epoch 3] cost = 0.046288877725601196
[Epoch 4] cost = 0.03746114671230316
[Epoch 5] cost = 0.031336210668087006
[Epoch 6] cost = 0.0260468702763319
[Epoch 7] cost = 0.021909423172473907
[Epoch 8] cost = 0.018533028662204742
[Epoch 9] cost = 0.01640661433339119
[Epoch 10] cost = 0.013376807793974876
[Epoch 11] cost = 0.010003098286688328
[Epoch 12] cost = 0.010247739963233471
[Epoch 13] cost = 0.008195456117391586
[Epoch 14] cost = 0.00696233194321394
[Epoch 15] cost = 0.006207533646374941
Learning Finished!


In [83]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1,28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy:", accuracy.item())

Accuracy: 0.9887999892234802
